In [3]:
from sympy import *

In [5]:
# Определим функции от T:
# Pjx - x координата подшипника
# Pjy - y координата подшипника
# FAlpha - угол наклонения шеста
Pjx, Pjy, FAlpha = symbols('P_{jx} P_{jy} alpha', cls=Function)


# Определим переменные:
# Alpha - угол наклонения шеста (смотрите ниже, зачем второй угол)
# T - время
# L - длина шеста
# Ajx - ускорение подшипника по X
Alpha, T, L, Ajx = symbols('alpha t l A_{jx}')

Выразим координаты центра масс шеста через угол и положение подшипника:

In [7]:
p_cx = Pjx(T) + L/2 * sin(FAlpha(T))
p_cy = Pjy(T) + L/2 * cos(FAlpha(T))

In [10]:
# A_alpha - угловое ускорение
# Omega - угловая скорость
A_alpha, Omega = symbols('A_{alpha} omega')

Теперь посчитаем $$\frac{d^2p_{cx}}{dT^2}$$ и 
$$\frac{d^2p_{cy}}{dT^2}$$

In [84]:
a_cx, a_cy = map(lambda e: e.diff(T).diff(T), [p_cx, p_cy])

In [85]:
a_cx

-l*sin(alpha(t))*Derivative(alpha(t), t)**2/2 + l*cos(alpha(t))*Derivative(alpha(t), (t, 2))/2 + Derivative(P_{jx}(t), (t, 2))

In [86]:
a_cy

-l*sin(alpha(t))*Derivative(alpha(t), (t, 2))/2 - l*cos(alpha(t))*Derivative(alpha(t), t)**2/2 + Derivative(P_{jy}(t), (t, 2))

Для удобства заменим $\frac{d^2\alpha(t)}{dt^2}$ на $A_\alpha$, $\frac{d\alpha(t)}{dt}$ на $\omega$, $\alpha(t)$ на $\alpha$,  
$\frac{d^2P_{jx}}{dt^2}$ на $A_{jx}$

Еще заметим, что Y-ускорение подшипника равно 0: $\frac{d^2P_{jy}}{dt^2} = 0$

In [87]:
def stage1(e):
    e = e.subs(FAlpha(T).diff(T).diff(T), A_alpha)
    e = e.subs(FAlpha(T).diff(T), Omega)
    e = e.subs(FAlpha(T), Alpha)
    e = e.subs(Pjx(T).diff(T).diff(T), Ajx)
    e = e.subs(Pjy(T).diff(T).diff(T), 0)
    return e

a_cx, a_cy = map(stage1, [a_cx, a_cy])

In [88]:
a_cx

A_{alpha}*l*cos(alpha)/2 + A_{jx} - l*omega**2*sin(alpha)/2

In [89]:
a_cy

-A_{alpha}*l*sin(alpha)/2 - l*omega**2*cos(alpha)/2

Запишем динамику центра масс шеста:

In [90]:
#G, M_pole, Ajx, Ajy, Tx, Ty = sympy.symbols('G M_{pole} A_{jx} A_{jy} T_x T_y')
G, M_pole, Tx, Ty = symbols('G M_{pole} T_x T_y')
eq0 = Eq(Tx, M_pole * a_cx)
eq1 = Eq(Ty - G*M_pole, M_pole * a_cy)

Сила Tx вызывает ускорение a_cx:

In [91]:
eq0

Eq(T_x, M_{pole}*(A_{alpha}*l*cos(alpha)/2 + A_{jx} - l*omega**2*sin(alpha)/2))

Сила Ty совместно с силой тяжести определяет ускорение a_cy:

In [92]:
eq1

Eq(-G*M_{pole} + T_y, M_{pole}*(-A_{alpha}*l*sin(alpha)/2 - l*omega**2*cos(alpha)/2))

X-ускорение подшипника равно ускорению машинки и вызвано внешней силой и силой реакции Tx в подшипнике:

In [93]:
F, M_car = symbols('F M_{car}')
eq2 = Eq(M_car * Ajx, F - Tx)
eq2

Eq(A_{jx}*M_{car}, F - T_x)

Свяжем угловое ускорение и момент сил:

In [94]:
eq3 = Eq(M_pole*L**2 / 12 * A_alpha, 1/2 * L * (Ty * sin(Alpha) - Tx * cos(Alpha)))
eq3

Eq(A_{alpha}*M_{pole}*l**2/12, 0.5*l*(-T_x*cos(alpha) + T_y*sin(alpha)))

Подставим в уравнение eq3 выражения для сил реакций в подшипнике, выраженных через уравнения eq0 и eq1

In [95]:
eq4 = eq3.subs(Tx, solve(eq0, Tx)[0]).subs(Ty, solve(eq1, Ty)[0])
eq4

Eq(A_{alpha}*M_{pole}*l**2/12, 0.5*l*(M_{pole}*(-A_{alpha}*l*sin(alpha) + 2*G - l*omega**2*cos(alpha))*sin(alpha)/2 - M_{pole}*(A_{alpha}*l*cos(alpha) + 2*A_{jx} - l*omega**2*sin(alpha))*cos(alpha)/2))

Аналогично в уравнении eq2 избавимся от Tx:

In [96]:
eq5 = eq2.subs(Tx, solve(eq0, Tx)[0])
eq5

Eq(A_{jx}*M_{car}, F - M_{pole}*(A_{alpha}*l*cos(alpha) + 2*A_{jx} - l*omega**2*sin(alpha))/2)

Решим eq4 и eq5 и найдем ускорение подшипника и угловое ускорение:

In [97]:
r = solve([eq4, eq5], [Ajx, A_alpha])

Ускорение подшипника:

In [98]:
r[Ajx]

(4.0*F - 1.5*G*M_{pole}*sin(2.0*alpha) + 2.0*M_{pole}*l*omega**2*sin(alpha))/(4.0*M_{car} + 3.0*M_{pole}*sin(alpha)**2 + M_{pole})

Угловое ускорение:

In [100]:
r[A_alpha]

(6.0*G*(M_{car} + M_{pole})*sin(alpha) - 3.0*(2.0*F + M_{pole}*l*omega**2*sin(alpha))*cos(alpha))/(l*(4.0*M_{car} + 3.0*M_{pole}*sin(alpha)**2 + M_{pole}))